In [1]:


import sys
import json
import pickle
from transformers import (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)

/Users/mahbubcseju/opt/anaconda3/envs/codebert/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.path.append('/Users/mahbubcseju/Desktop/projects/vul_detect')

In [3]:
with open("../../data/devign/test.jsonl", 'r') as f:
    data = []
    for  line in f:
        try:
            data.append(json.loads(line.strip()))
        except Exception as e:
            print(e)


In [4]:
config_class, model_class, tokenizer_class = RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer


In [5]:
tokenizer = tokenizer_class.from_pretrained('microsoft/codebert-base')

In [6]:
explanations = pickle.load(open('explanation_devign.pkl', 'rb'))

In [7]:
assert len(explanations) == len(data)

In [8]:
def get_max_match(original, start_index,  tokens):
    lo = len(original)
    lt = len(tokens)
    ind , max_score = 0, lo
    for i in range(start_index, lo-lt+ 1):
        found = True
        for j in range(lt):
            if  original[i + j] != tokens[j]:
                found = False
        if found:
            return [x for x in range(i, i + lt)]
    return []

In [9]:
def calculate_score(scores, ids):
    sum = 0
    for id in ids:
        sum += scores[id]
    return sum

In [11]:
results = []

for sample, explanation in zip(data, explanations):
    # print(sample, explanation)
    original_tokens = ['<s>'] + tokenizer.tokenize(sample['func'])[:398] + ['</s>']
    explanation_tokens = explanation['token_grad_sentence'].tokens
    explanation_scores = explanation['token_grad_sentence'].salience
    if original_tokens != explanation_tokens:
        print(sample['idx'])
    
    lines = sample['func'].split("\n")
    # print(len(lines))
    # while lines.count(''):
    #     lines.remove('')
    # print(lines[0])
    # print(tokenizer.tokenize(lines[0]))
    # print(explanation_tokens)
    # print(explanation_scores)
    start_index = 0
    lines_scores = []
    cnt = 0
    for line in lines:
        line_tokens = tokenizer.tokenize(line)
        ids = get_max_match(explanation_tokens, start_index, line_tokens)
        if  ids:
            start_index = ids[-1] + 1
            cnt += 1
        # print(line_tokens)
        # print(ids)
        # print(start_index)
        lines_scores.append(calculate_score(explanation_scores, ids))
    # print(len(lines), len(lines_scores))
    # break
    print(lines_scores)
    results.append({
        'idx': sample['idx'],
        'scores': lines_scores,
    })


[0.4242668066872284, 0, 0.04313964991160901, 0, 0.009404022246599197, 0, 0.005717527266824618, 0, 0.009260383871151134, 0, 0, 0, 0.007210499526991043, 0, 0.0713274579757126, 0, 0.01130852394271642, 0, 0.07792205386795104, 0, 0, 0, 0.03676784111303277, 0, 0.004091009439434856, 0, 0.031740240645376616, 0, 0.002738856062933337, 0, 0.02488851693487959, 0, 0.016103097958875878, 0, 0.018437073603308818, 0, 0.018837819296095404, 0, 0.0009172512036457192, 0, 0.000976949326286558, 0, 0.01431579379550385, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

KeyboardInterrupt: 

In [22]:
json.dump(results, open("line_scores.json", 'w'))